In [1]:
import tqdm
import numpy as np
import matplotlib.pyplot as plt
from centrex_tlf import states, hamiltonian

In [2]:
from decimal import Decimal


def fexp(number):
    (sign, digits, exponent) = Decimal(number).as_tuple()
    return len(digits) + exponent - 1

# Zero field
## Ω basis

In [3]:
QNc = states.generate_coupled_states_excited(Js=[1, 2, 3, 4], Ps=None, Omegas=[-1, 1])
qn_select = states.QuantumSelector(
    J=1, F1=1 / 2, F=1, electronic=states.ElectronicState.B
)
QNc[qn_select.get_indices(QNc)]

array([|B, J = 1, F₁ = 1/2, F = 1, mF = -1, I₁ = 1/2, I₂ = 1/2, Ω = -1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = -1, I₁ = 1/2, I₂ = 1/2, Ω = 1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = 0, I₁ = 1/2, I₂ = 1/2, Ω = -1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = 0, I₁ = 1/2, I₂ = 1/2, Ω = 1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = 1, I₁ = 1/2, I₂ = 1/2, Ω = -1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = 1, I₁ = 1/2, I₂ = 1/2, Ω = 1>],
      dtype=object)

In [4]:
# generate the X hamiltonian terms
H = hamiltonian.generate_coupled_hamiltonian_B(QNc)

# create a function outputting the hamiltonian as a function of E and B
Hfunc = hamiltonian.generate_coupled_hamiltonian_B_function(H)

# generate the Hamiltonian for (almost) zero field, add a small field to make states
# non-degenerate
Hi = Hfunc(E=[0, 0, 0], B=[0, 0, 1e-5])
E, V = np.linalg.eigh(Hi)

# get the true superposition-states of the system
QN_states = hamiltonian.matrix_to_states(V, QNc)

In [5]:
QN_states_omega, E_omega = QN_states, E

## P basis

In [6]:
QNc = states.generate_coupled_states_excited(Js=[1, 2, 3, 4], Ps=[-1, 1], Omegas=1)
qn_select = states.QuantumSelector(
    J=1, F1=1 / 2, F=1, electronic=states.ElectronicState.B
)
QNc[qn_select.get_indices(QNc)]

array([|B, J = 1, F₁ = 1/2, F = 1, mF = -1, I₁ = 1/2, I₂ = 1/2, P = -, Ω = 1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = -1, I₁ = 1/2, I₂ = 1/2, P = +, Ω = 1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = 0, I₁ = 1/2, I₂ = 1/2, P = -, Ω = 1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = 0, I₁ = 1/2, I₂ = 1/2, P = +, Ω = 1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = 1, I₁ = 1/2, I₂ = 1/2, P = -, Ω = 1>,
       |B, J = 1, F₁ = 1/2, F = 1, mF = 1, I₁ = 1/2, I₂ = 1/2, P = +, Ω = 1>],
      dtype=object)

In [7]:
# generate the X hamiltonian terms
H = hamiltonian.generate_coupled_hamiltonian_B(QNc)

# create a function outputting the hamiltonian as a function of E and B
Hfunc = hamiltonian.generate_coupled_hamiltonian_B_function(H)

# generate the Hamiltonian for (almost) zero field, add a small field to make states
# non-degenerate
Hi = Hfunc(E=[0, 0, 0], B=[0, 0, 1e-5])
E, V = np.linalg.eigh(Hi)

# get the true superposition-states of the system
QN_states = hamiltonian.matrix_to_states(V, QNc)

## Check if the resulting states are the same for the P and Ω basis

In [8]:
# at a tolerance of 1e-5 some states will have slightly different coefficients between
# the two bases
tol = 1e-4

for qnp, qno in tqdm.tqdm(zip(QN_states, QN_states_omega), total=len(QN_states)):
    qnp = qnp.remove_small_components(tol)
    qno = qno.transform_to_parity_basis().remove_small_components(tol)
    qno.data = [(np.round(a, abs(fexp(tol))), s) for a, s in qno.data]
    qnp.data = [(np.round(a, abs(fexp(tol))), s) for a, s in qnp.data]
    if qnp != qno and qnp != -qno:
        print(qno)
        print(qnp)
        raise AssertionError("Mismatch between different bases.")

100%|██████████| 192/192 [00:06<00:00, 27.65it/s]


# Non-zero electric field
## Ω basis

In [9]:
QNc = states.generate_coupled_states_excited(Js=[1, 2, 3, 4], Ps=None, Omegas=[-1, 1])

# generate the X hamiltonian terms
H = hamiltonian.generate_coupled_hamiltonian_B(QNc)

# create a function outputting the hamiltonian as a function of E and B
Hfunc = hamiltonian.generate_coupled_hamiltonian_B_function(H)

# generate the Hamiltonian for (almost) zero field, add a small field to make states
# non-degenerate
Hi = Hfunc(E=[0, 0, 200], B=[0, 0, 1e-5])
E, V = np.linalg.eigh(Hi)

# get the true superposition-states of the system
QN_states = hamiltonian.matrix_to_states(V, QNc)

QN_states_omega, E_omega = QN_states, E

## P basis

In [10]:
QNc = states.generate_coupled_states_excited(Js=[1, 2, 3, 4], Ps=[-1, 1], Omegas=1)

# generate the X hamiltonian terms
H = hamiltonian.generate_coupled_hamiltonian_B(QNc)

# create a function outputting the hamiltonian as a function of E and B
Hfunc = hamiltonian.generate_coupled_hamiltonian_B_function(H)

# generate the Hamiltonian for (almost) zero field, add a small field to make states
# non-degenerate
Hi = Hfunc(E=[0, 0, 200], B=[0, 0, 1e-5])
E, V = np.linalg.eigh(Hi)

# get the true superposition-states of the system
QN_states = hamiltonian.matrix_to_states(V, QNc)

## Check if the resulting states are the same for the P and Ω basis

In [11]:
tol = 1e-4
QN_states_filtered = [s.remove_small_components(tol) for s in QN_states]

indices_check = []
for qno in tqdm.tqdm(QN_states_omega):
    qno = qno.transform_to_parity_basis().remove_small_components(tol)
    vector = np.abs(qno.state_vector(QN_states_filtered))
    assert np.argmax(vector) not in indices_check
    indices_check.append(np.argmax(vector))

100%|██████████| 192/192 [00:21<00:00,  9.14it/s]


In [12]:
# at a tolerance of 1e-5 some states will have slightly different coefficients between
# the two bases
tol = 1e-4
for qnp, qno in tqdm.tqdm(zip(QN_states, QN_states_omega), total=len(QN_states)):
    qnp = qnp.order_by_amp().remove_small_components(tol)
    qno = qno.transform_to_parity_basis().order_by_amp().remove_small_components(tol)
    qno.data = [(np.round(a, abs(fexp(tol))), s) for a, s in qno.data]
    qnp.data = [(np.round(a, abs(fexp(tol))), s) for a, s in qnp.data]
    if qnp != qno and qnp != -qno:
        print(qno)
        print(qnp)
        raise AssertionError("Mismatch between different bases.")

100%|██████████| 192/192 [00:07<00:00, 24.20it/s]
